In [22]:
import pandas as pd
import numpy as np
import requests
import os

## Pandas 

#### **Question 1.1: The ML Dataset Preprocessor**

**Scenario**: You just downloaded a messy dataset for your first ML project. It's exactly the kind of data you'll face in real ML work.

**Your Mission**: Create a dataset with 1000 rows and these columns:

- `customer_id`: 1 to 1000
- `age`: random integers 18-80, but 50 values are missing (NaN)
- `income`: random floats 20000-100000, but 30 values are negative (data errors)
- `category`: random choice from ['A', 'B', 'C', 'D'], but 40 values are empty strings
- `purchase_amount`: random floats 10-5000
- `date`: random dates in 2023, but 25 dates are in wrong format (strings like "2023/13/45")

**Detailed Question**: How do you clean this dataset to make it ML-ready? You need to:

- Handle missing ages (should you drop, fill with mean, or median?)
- Fix negative incomes (convert to positive or remove?)
- Handle empty categories (fill with 'Unknown' or most frequent?)
- Fix invalid dates (convert or remove?)
- Create new features: age_group (Young/Middle/Senior), income_level (Low/Medium/High)

**Hint**: Use `pd.DataFrame()` with random data, `np.nan`, `pd.to_datetime()`, `pd.cut()` for binning, and conditional logic for cleaning.


In [23]:
n = 1000
df = pd.DataFrame(
    {
        "customer_id" : np.arange(1,n+1),
        "age" : np.random.randint(18,81,size=n),
        "income" : np.random.uniform(20000.0,100000.0, size=n),
        "category" : np.random.choice(["A","B","C","D"], size=n),
        "purchase_amount" : np.random.uniform(10.0,5000.0, size=n),
        "date": pd.to_datetime(np.random.randint(
        pd.Timestamp("2023-01-01").value, 
        pd.Timestamp("2024-01-01").value, 
        size=n)
    )
    }
)

In [24]:
missing_age_indices = np.random.choice(n,size=50, replace=False) #replace one is only require to prevent repeat
df.loc[missing_age_indices, 'age'] = np.nan

negative_income_indices = np.random.choice(n,size=30, replace=False) 
df.loc[negative_income_indices, 'income'] = -1

empty_string_indices = np.random.choice(n,size=40, replace = False)
df.loc[empty_string_indices, 'category'] = ""

In [25]:
sf = df[df['age'].notna()]
m = sf['age'].mean()
df['age'] = df['age'].fillna(value=m)

df.loc[df['income'] < 0, 'income'] = np.nan
df = df[df['income'].notna()]

freq = df["category"].value_counts().idxmax()
df.loc[df['category'] == "", 'category'] = freq




In [26]:
bins1 = [18,30,55,80]
bins2 = [20000,45000,70000,100000]

labels1 = ["Young", "Middle", "Senior"]
labels2 = ["Low", "Medium", "High"]

age_groups = pd.cut(df['age'], bins=bins1, labels=labels1)
income_level = pd.cut(df['income'], bins=bins2, labels=labels2)

df["age_groups"] = age_groups
df["income_level"] = income_level

In [27]:
print(df)

     customer_id   age        income category  purchase_amount  \
0              1  27.0  77057.425624        D       610.112747   
1              2  69.0  89279.367991        B      3083.721170   
2              3  71.0  71678.546157        D      3796.214683   
3              4  67.0  91848.496448        A       222.834355   
4              5  38.0  98014.377415        D      1573.337333   
..           ...   ...           ...      ...              ...   
995          996  61.0  67117.474695        B      2476.708899   
996          997  74.0  99980.107224        B       770.726820   
997          998  41.0  97314.696722        D      4904.127206   
998          999  38.0  91718.167282        D        13.909988   
999         1000  42.0  96180.616296        C        18.055443   

                             date age_groups income_level  
0   2023-01-06 02:05:35.032099792      Young         High  
1   2023-09-15 02:05:12.742059989     Senior         High  
2   2023-08-06 06:42:40.235

#### **Question 1.2: The Time Series Explorer**

**Scenario**: You're analyzing user behavior data for an ML model that predicts user churn.

**Create this dataset**:

```python
# 365 days of user activity data
dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
users = ['user_' + str(i) for i in range(1, 101)]  # 100 users
```

**Detailed Question**: Generate a dataset where each user has daily activity data (login_count, session_duration, clicks). Then:

- Find users with the highest average activity per month
- Identify users who were active early in the year but stopped (potential churn)
- Calculate 7-day and 30-day rolling averages for each user
- Find the most active day of the week across all users
- Create a "user_risk_score" based on recent activity decline

**Hint**: Use `pd.MultiIndex`, `groupby()`, `rolling()`, `resample()`, and custom functions for risk calculation.


In [28]:
dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
users = ['user_' + str(i) for i in range(1, 101)] 
arrays = [users,dates]
index = pd.MultiIndex.from_product(arrays, names=["user","date"])

In [31]:
df = pd.DataFrame({
    "Activity": np.random.randint(1,16,size=36500)
}, index=index).reset_index()

df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')
df['week'] = df['date'].dt.isocalendar().week

In [ ]:
monthly_avg = df.groupby(['user','month'])['Activity'].mean().reset_index()
top_users = monthly_avg.loc[monthly_avg.groupby('month')['Activity'].idxmax()]

last_active_date = df[df['Activity'] > 0].groupby('user')['date'].max().reset_index()
churned_users = last_active_date[last_active_date['date'] < '2023-09-01']

df = df.sort_values(['user', 'date'])
df['rolling_7'] = df.groupby('user')['Activity'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['rolling_30'] = df.groupby('user')['Activity'].transform(lambda x: x.rolling(30, min_periods=1).mean())

df['day_name'] = df['date'].dt.day_name()
weekday_avg = df.groupby('day_name')['Activity'].mean().sort_values(ascending=False)
most_active_day = weekday_avg.idxmax()

risk_scores = []
for user in df['user'].unique():
    user_df = df[df['user'] == user]
    first_30 = user_df.head(30)['Activity'].mean()
    last_30 = user_df.tail(30)['Activity'].mean() 
    score = last_30 / (first_30 + 1e-6)  # Add epsilon to avoid division by zero
    risk_scores.append((user, score))
risk_df = pd.DataFrame(risk_scores, columns=['user', 'user_risk_score'])
# Sort: lowest score = highest risk
risk_df = risk_df.sort_values('user_risk_score')

